In [8]:
import numpy as np
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score, roc_auc_score, matthews_corrcoef
import os


test_path = ''

df_test = pd.read_csv(test_path)
df_test = df_test.fillna(df_test.median())
X_test_raw = df_test.iloc[:, :-1].values
y_test = df_test.iloc[:, -1].values

scaler = joblib.load('feature_scaler.pkl')
X_test_scaled = scaler.transform(X_test_raw)

selected_indices = joblib.load('lasso_selected_indices.pkl')
X_test_selected = X_test_scaled[:, selected_indices]

base_model_names = ['xgb','rf','mlp','et','gbdt']
base_models = {}

from sklearn.metrics import precision_score, f1_score

for name in base_model_names:
    model_path = f''
    if os.path.exists(model_path):
        model = joblib.load(model_path)
        base_models[name] = model

        y_pred = model.predict(X_test_selected)
        y_proba = model.predict_proba(X_test_selected)[:, 1]

        TP = np.sum((y_test == 1) & (y_pred == 1))
        TN = np.sum((y_test == 0) & (y_pred == 0))
        FP = np.sum((y_test == 0) & (y_pred == 1))
        FN = np.sum((y_test == 1) & (y_pred == 0))

        Sn = TP / (TP + FN) if TP + FN > 0 else 0
        Sp = TN / (TN + FP) if TN + FP > 0 else 0
        Acc = accuracy_score(y_test, y_pred)
        Pre = precision_score(y_test, y_pred, zero_division=0)
        F1 = f1_score(y_test, y_pred, zero_division=0)
        AUC = roc_auc_score(y_test, y_proba)

        print(f"[{name.upper()}] Sn: {Sn:.4f} | Sp: {Sp:.4f} | Acc: {Acc:.4f} | Pre: {Pre:.4f} | F1: {F1:.4f} | AUC: {AUC:.4f}")
    else:
        print(f"Model file was not found: {model_path}")

meta_clf = joblib.load('')

meta_input = np.zeros((X_test_selected.shape[0], len(base_models)))
for i, name in enumerate(base_model_names):
    if name in base_models:
        proba = base_models[name].predict_proba(X_test_selected)[:, 1]
        meta_input[:, i] = proba

y_pred_meta = meta_clf.predict(meta_input)
y_proba_meta = meta_clf.predict_proba(meta_input)[:, 1]

TP = np.sum((y_test == 1) & (y_pred_meta == 1))
TN = np.sum((y_test == 0) & (y_pred_meta == 0))
FP = np.sum((y_test == 0) & (y_pred_meta == 1))
FN = np.sum((y_test == 1) & (y_pred_meta == 0))

Sn = TP / (TP + FN) if TP + FN > 0 else 0
Sp = TN / (TN + FP) if TN + FP > 0 else 0
Acc = accuracy_score(y_test, y_pred_meta)
Pre = precision_score(y_test, y_pred_meta, zero_division=0)
F1 = f1_score(y_test, y_pred_meta, zero_division=0)
AUC = roc_auc_score(y_test, y_proba_meta)

print(f"\n[META] Sn: {Sn:.4f} | Sp: {Sp:.4f} | Acc: {Acc:.4f} | Pre: {Pre:.4f} | F1: {F1:.4f} | AUC: {AUC:.4f}")


[XGB] Sn: 0.8071 | Sp: 0.9357 | Acc: 0.9036 | Pre: 0.8071 | F1: 0.8071 | AUC: 0.9358
[RF] Sn: 0.8571 | Sp: 0.9310 | Acc: 0.9125 | Pre: 0.8054 | F1: 0.8304 | AUC: 0.9277
[MLP] Sn: 0.8143 | Sp: 0.9214 | Acc: 0.8946 | Pre: 0.7755 | F1: 0.7944 | AUC: 0.9378
[ET] Sn: 0.7857 | Sp: 0.9381 | Acc: 0.9000 | Pre: 0.8088 | F1: 0.7971 | AUC: 0.9398
[GBDT] Sn: 0.8429 | Sp: 0.9238 | Acc: 0.9036 | Pre: 0.7867 | F1: 0.8138 | AUC: 0.9310

[META] Sn: 0.8643 | Sp: 0.9286 | Acc: 0.9125 | Pre: 0.8013 | F1: 0.8316 | AUC: 0.9339
